# 機械特性を計算するための準備

## Quantum espressoでSiO2の機械特性を計算する

In [1]:
from mlptools.io.write import QuantumEspressoWriter
from ase.io.vasp import read_vasp

In [2]:
path2atom = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-559091/original/POSCAR"
atoms = read_vasp(path2atom)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-559091/original/POSCAR'

In [ ]:
atoms.cell[:]

array([[ 2.53453694, -4.38994676,  0.        ],
       [ 2.53453694,  4.38994676,  0.        ],
       [ 0.        ,  0.        ,  8.55848854]])

In [ ]:
for symbol, scaled_position in zip(atoms.get_chemical_symbols(), atoms.get_scaled_positions()):
    print(" ".join([symbol] + list(map(str, scaled_position))))

Si 0.6666666666666666 0.3333333333333333 0.9371258
Si 0.6666666666666666 0.33333333333333337 0.5628742
Si 0.3333333333333335 0.6666666666666667 0.0628742
Si 0.33333333333333337 0.6666666666666666 0.4371258
O 0.0 0.4212587400000001 0.0
O 0.4212587400000001 0.4212587400000001 0.5
O 0.5787412599999999 0.0 0.5
O 0.5787412599999999 0.5787412599999999 0.0
O 0.0 0.5787412599999999 0.5
O 0.6666666666666666 0.3333333333333333 0.7500000000000001
O 0.33333333333333337 0.6666666666666666 0.25
O 0.4212587400000001 0.0 0.0


# n2p2で訓練後に各epochの機械特性を計算する

In [3]:
import os
from glob import glob
import shutil
from typing import List
from mlptools.utils.utils import remove_empty_from_array

path2n2p2_result = '/home/y1u0d2/result/n2p2/Si/102/03'
# path2template = '/home/y1u0d2/result/lammps/scripts/SiO2/elastic/template'
path2template = '/home/y1u0d2/result/lammps/scripts/Si/elastic/template'


def change_nnpdir(lines, path2nnpdir):
    for i, l in enumerate(lines):
        if 'variable nnpDir' in l:
            dir_l = remove_empty_from_array(l.split(' '))
            dir_l[-1] = f'"{path2nnpdir}"'
            lines[i] = ' '.join(dir_l)
            break
    return lines


def setup_mechanical_props_calc(path2target: str ,atomic_number_list: List[int]):
    path2elastic = os.path.join(path2target, 'elastic')
    if not os.path.exists(path2elastic):
        print(f"Create directory {path2elastic}")
        os.mkdir(path2elastic)

    atomic_number_list_preprocessed = [str(n).zfill(3) for n in atomic_number_list]
    weights_file_dict = {}
    for atomic_number_str in atomic_number_list_preprocessed:
        weights_file_dict[atomic_number_str] = sorted(glob(f'{path2n2p2_result}/weights.{atomic_number_str}.*'))

    for i in range(len(weights_file_dict[atomic_number_str])):
        weights_file_list = []
        for _, values in weights_file_dict.items():
            weights_file_list.append(values[i])
        
        epoch_set = {int(w.split('/')[-1].split('.')[-2]) for w in weights_file_list}
        if len(epoch_set) != 1:
            raise ValueError('epoch is not consistent')
        epoch = epoch_set.pop()
        print(f"epoch: {epoch}")
        path2epoch = os.path.join(path2elastic, f'e_{epoch}')

        # copy mechanical prop files
        if not os.path.exists(path2epoch): shutil.copytree(path2template, path2epoch)

        # copy n2p2 files
        shutil.copy(os.path.join(path2n2p2_result, 'input.nn'), path2epoch)
        shutil.copy(os.path.join(path2n2p2_result, 'scaling.data'), path2epoch)
        for weights_f in weights_file_list:
            shutil.copy(weights_f, path2epoch)
            print(f"[COPY] {weights_f} to {path2epoch}")
            atomic_str = os.path.basename(weights_f).split('.')[1]
            print(f"[RENAME] {os.path.basename(weights_f)} to weights.{atomic_str}.data")
            os.rename(os.path.join(path2epoch, os.path.basename(weights_f)), os.path.join(path2epoch, f"weights.{atomic_str}.data"))

        # change potential.mod
        with open(os.path.join(path2epoch, 'potential.mod'), mode='r') as f:
            lines = [s.strip() for s in f.readlines()]
        changed_lines = change_nnpdir(lines, path2epoch)
        with open(os.path.join(path2epoch, 'potential.mod'), mode='w') as f:
            f.write('\n'.join(changed_lines))

In [4]:
setup_mechanical_props_calc(
    path2target=path2n2p2_result,
    atomic_number_list=[14]
)

Create directory /home/y1u0d2/result/n2p2/Si/102/03/elastic
epoch: 0
[COPY] /home/y1u0d2/result/n2p2/Si/102/03/weights.014.000000.out to /home/y1u0d2/result/n2p2/Si/102/03/elastic/e_0
[RENAME] weights.014.000000.out to weights.014.data
epoch: 1
[COPY] /home/y1u0d2/result/n2p2/Si/102/03/weights.014.000001.out to /home/y1u0d2/result/n2p2/Si/102/03/elastic/e_1
[RENAME] weights.014.000001.out to weights.014.data
epoch: 2
[COPY] /home/y1u0d2/result/n2p2/Si/102/03/weights.014.000002.out to /home/y1u0d2/result/n2p2/Si/102/03/elastic/e_2
[RENAME] weights.014.000002.out to weights.014.data
epoch: 3
[COPY] /home/y1u0d2/result/n2p2/Si/102/03/weights.014.000003.out to /home/y1u0d2/result/n2p2/Si/102/03/elastic/e_3
[RENAME] weights.014.000003.out to weights.014.data
epoch: 4
[COPY] /home/y1u0d2/result/n2p2/Si/102/03/weights.014.000004.out to /home/y1u0d2/result/n2p2/Si/102/03/elastic/e_4
[RENAME] weights.014.000004.out to weights.014.data
epoch: 5
[COPY] /home/y1u0d2/result/n2p2/Si/102/03/weights.0